# QTM 151 - Introduction to Statistical Computing II
## Lecture 12 - SQLite in Python
**Author:** Danilo Freire (danilo.freire@emory.edu, Emory University)

# Hello! Great to see you again! 😊

# Lecture overview 📚

## Lecture overview

### Last time we learned: 

- What are relational databases and why they are still important in data science 
- How to create databases, tables, and insert data using basic SQL commands 
- How to perform queries with `SELECT`, `WHERE` and `ORDER BY` 
- Group data with `GROUP BY` and filter groups with `HAVING` 
- Statistics with `COUNT`, `SUM`, `AVG`, `MIN`, and `MAX` 
- That was a nice revision! 🤓 
- SQLite is generally easier to set up as it's often file-based or in-memory

### Today we will learn: 

- How to connect SQLite with Python
- We will use the built-in `sqlite3` library together with `pandas` 
- Other SQL commands, such as `LIKE`, `IN`, `BETWEEN`, `CASE`, window functions, and string functions like `SUBSTR` and `LENGTH` 
- How to fill missing data with `COALESCE` 
- How to use database cursors and fetch results (`fetchall`, `fetchone`) 
- If time allows, we will also see how to pivot tables using `CASE`
- Let's get started! 🚀

# Connecting SQLite with Python 🐍

## Connecting and Creating a Cursor

- To interact with an SQLite database from Python, we first need to establish a connection and then create a cursor.
- The **connection** links our script to the database file.
- The **cursor** is like a handle we use to send SQL commands and manage results.

In [ ]:
import sqlite3

db_file = 'lecture12.db' # Database filename

# 1. Connect to the database (creates file if needed)
conn = sqlite3.connect(db_file) 

# 2. Create a cursor object to execute commands
cur = conn.cursor()

# Executing SQL in Python

## Creating Tables

- We define the structure of our data using `CREATE TABLE`.
- Use `cursor.execute()` to run the command and `conn.commit()` to **save** the change.

In [ ]:
# Use triple quotes for multi-line SQL
# Drop table if exist 
sql_drop = '''
DROP TABLE IF EXISTS drivers; 
'''

# Execute the SQL command
cur.execute(sql_drop)

# Create a new table named 'drivers'
sql_create = '''
CREATE TABLE drivers (
    driver_id INTEGER PRIMARY KEY AUTOINCREMENT,
    driver_name TEXT,
    team TEXT,
    nationality TEXT,
    victories INTEGER
);
'''
# Execute the SQL command
cur.execute(sql_create)

# Commit the transaction to save the table
conn.commit() 

# Check if the table was created
cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='drivers';")
print(cur.fetchall()) # Should show [('drivers',)] if successful

## Inserting Data

- Once the table exists, we populate it using `INSERT INTO`.
- Again, `cursor.execute()` runs the command, and `conn.commit()` **makes the insertions permanent**.

In [ ]:
# Insert a few rows
cur.execute("INSERT INTO drivers (driver_name, team, nationality, victories) VALUES ('Lewis Hamilton', 'Mercedes', 'British', 103);")
cur.execute("INSERT INTO drivers (driver_name, team, nationality, victories) VALUES ('Max Verstappen', 'Red Bull Racing', 'Dutch', 55);")
cur.execute("INSERT INTO drivers (driver_name, team, nationality, victories) VALUES ('Fernando Alonso', 'Aston Martin', NULL, NULL);")
cur.execute("INSERT INTO drivers (driver_name, team, nationality) VALUES ('Charles Leclerc', 'Ferrari', 'Monégasque');") 

# Commit the insertions
conn.commit()

## Fetching All Results (`fetchall`) and One Result (`fetchone`)

- After a `SELECT` query, `fetchall()` retrieves **all results at once** into a list of tuples.
- Useful if you need the full dataset immediately, but can use more memory for large results.

In [ ]:
cur.execute('SELECT driver_id, driver_name, team FROM drivers') 
all_rows = cur.fetchall() # Get the list of tuples

# Print the fetched data
for row in all_rows:
    print(row)

- `fetchone()` retrieves rows **one by one**. Each call gets the next available row.
- Returns `None` when no more rows are left. Good for processing sequentially or just getting the top item.

In [ ]:
cur.execute('SELECT driver_id, driver_name FROM drivers ORDER BY driver_id')

row1 = cur.fetchone()
print("First row:", row1) 

row2 = cur.fetchone()
print("Second row:", row2)

## Iterating Over the Cursor

- A more "Pythonic" and **memory-efficient** way to process results row-by-row is to iterate directly over the cursor.

In [ ]:
# Execute the query
query = "SELECT driver_name, team FROM drivers WHERE driver_name LIKE 'M%'"

# Loop directly over the cursor object after execute
print("\nDrivers starting with 'M' (iterating cursor):")
for row_tuple in cur.execute(query):
    print(row_tuple)

# Filtering Data

## `IN` and `BETWEEN` Operators

- Use `IN` to check if a column's value matches **any value in a specified list**.

In [ ]:
query_in = "SELECT driver_name, team FROM drivers WHERE team IN ('Ferrari', 'Mercedes')" 
cur.execute(query_in)
print("Drivers in Ferrari or Mercedes:")
for row in cur.fetchall():
    print(row)

- `NOT IN` works similarly to exclude values in the list.

In [ ]:
query_not_in = "SELECT driver_name, team FROM drivers WHERE team NOT IN ('Ferrari', 'Mercedes')" 
cur.execute(query_not_in)
print("\nDrivers NOT in Ferrari or Mercedes:")
for row in cur.fetchall():
    print(row)

- `BETWEEN` checks if a value is within a specified range. **Inclusive** of the endpoints.

In [ ]:
query_between = "SELECT driver_name, victories FROM drivers WHERE victories BETWEEN 10 AND 60" 
cur.execute(query_between)
print("\nDrivers with victories between 10 and 60:")
for row in cur.fetchall():
    print(row)

- `NOT BETWEEN` excludes values within the specified range.

## `LIKE` Operator: Basic Patterns

- `LIKE` enables simple pattern matching in text strings.
  - **`%`**: Matches any sequence (including none) of characters.
  - **`_`**: Matches exactly one character.

In [ ]:
# Starts with 'L'
print("\nDrivers whose names start with 'L':")
cur.execute("SELECT driver_name FROM drivers WHERE driver_name LIKE 'L%'")
for row in cur.fetchall(): print(row)

In [ ]:
# Ends with 'Racing'
print("\nTeams ending with 'Racing':")
cur.execute("SELECT team FROM drivers WHERE team LIKE '%Racing'")
for row in cur.fetchall(): print(row)

In [ ]:
# 'o' as the second letter
print("\nDrivers with 'o' as the second letter:")
cur.execute("SELECT driver_name FROM drivers WHERE driver_name LIKE '_o%'")
for row in cur.fetchall(): print(row)

## `LIKE` Operator: Case Sensitivity and `NOT LIKE`

- SQLite `LIKE` is case-insensitive for ASCII by default. Use `LOWER()`/`UPPER()` for **reliable, explicit case handling**.

In [ ]:
# Case-insensitive search for names starting with 'l'
print("\nDrivers starting with 'l' (using LOWER()):")
cur.execute("SELECT driver_name FROM drivers WHERE LOWER(driver_name) LIKE 'l%'")
for row in cur.fetchall(): print(row)

- You can also use `COLLATE NOCASE` to make a specific comparison case-insensitive. **This is a more general (and recommended) approach.**

In [ ]:
# Case-insensitive search for names starting with 'l' using COLLATE
print("\nDrivers starting with 'l' (using COLLATE NOCASE):")
cur.execute("SELECT driver_name FROM drivers WHERE driver_name LIKE 'l%' COLLATE NOCASE")
for row in cur.fetchall(): print(row)

- Use `NOT LIKE` to find strings that **do not match** the pattern.

## Try it yourself! 🤓 {#sec:exercise01}

- Practice time! Use `cur.execute()` and fetching/looping.
- Find all drivers whose names start with 'M'.
- Find drivers whose nationality has exactly 7 characters (**`LENGTH()`** function might be useful).
- List drivers whose names start with 'L' **or** 'M'.
- Find drivers with 1 to 10 victories (**`BETWEEN`**).

In [ ]:
# Your SQL queries here, executed via cur.execute()

# Example for the first one:
# print("\n--- Drivers starting with M ---")
# cur.execute("SELECT driver_name FROM drivers WHERE driver_name LIKE 'M%'")
# for row in cur.fetchall():
#     print(row)

# print("\n--- Nationalities with 7 chars ---")
# # Your code here

# print("\n--- Drivers starting with L or M ---")
# # Your code here

# print("\n--- Drivers with 1-10 wins ---")
# # Your code here

# Handling Missing Data

## Finding NULLs (`IS NULL`)  and `COALESCE`

- SQL represents missing values as `NULL`. Use `IS NULL` to find them. **`IS NOT NULL` finds rows with values**.

In [ ]:
print("\nDrivers with NULL victories:")
cur.execute("SELECT driver_name, victories FROM drivers WHERE victories IS NULL")
for row in cur.fetchall(): print(row)

- `COALESCE(value1, value2, ...)` is useful for replacing `NULL`s with a default value. It returns the **first non-NULL expression** in its arguments.

In [ ]:
# If 'victories' is NULL, display 0 instead.
print("\nVictories (filled NULLs with 0):")
cur.execute("SELECT driver_name, COALESCE(victories, 0) AS victories_filled FROM drivers")
for row in cur.fetchall(): print(row)

## `COALESCE` with Subqueries

- The default value in `COALESCE` can be dynamic, calculated by a **subquery**.

In [ ]:
query_coalesce_sub = '''
SELECT 
  driver_name,
  COALESCE(victories, 
    -- Subquery calculates average victories from non-NULL rows
    (SELECT CAST(AVG(victories) AS INTEGER) 
     FROM drivers 
     WHERE victories IS NOT NULL) 
  ) AS victories_imputed
FROM drivers; 
''' 
print("\nVictories (imputed NULLs with overall average):")
cur.execute(query_coalesce_sub)
for row in cur.fetchall(): print(row)

- Here, `NULL` victories are replaced by the average of non-NULL victories.

# Window Functions

## Introduction to Window Functions

- Window functions compute values across a set of rows (a "window") related to the current row, without collapsing them like `GROUP BY`.
- Essential for ranking, running totals, moving averages, etc.
- Basic syntax involves the **`OVER()`** clause.
- **Requires SQLite version 3.25.0 or newer**.

In [ ]:
# Add more data for better examples
more_drivers_data = [
    ('Valtteri Bottas', 'Mercedes', 'Finnish', 10),
    ('Sergio Perez', 'Red Bull Racing', 'Mexican', 5),
    ('Lando Norris', 'McLaren', 'British', 2),
    ('Esteban Ocon', 'Alpine', 'French', 1) 
]
# Check if data already exists to avoid duplicates 
cur.execute("SELECT COUNT(*) FROM drivers WHERE driver_name = 'Valtteri Bottas'")
if cur.fetchone()[0] == 0:
    cur.executemany('INSERT INTO drivers (driver_name, team, nationality, victories) VALUES (?, ?, ?, ?)', more_drivers_data)
    conn.commit()
    print(f"Added {len(more_drivers_data)} more drivers.")
else:
    print("Additional drivers already exist.")


# Check SQLite Version
cur.execute("SELECT sqlite_version();")
print(f"SQLite Version: {cur.fetchone()[0]}")

## Window Functions: `AVG` and `RANK`

- Let's see common window functions in action:
- `AVG(...) OVER ()`: Average over the entire query result.
- `AVG(...) OVER (PARTITION BY col)`: Average within groups (partitions) defined by `col`.
- `RANK() OVER (ORDER BY col)`: Assigns rank based on `col` (**gaps possible for ties**).

In [ ]:
query_window_avg_rank = '''
SELECT 
    driver_name, team, victories,
    ROUND(AVG(victories) OVER (), 2) AS avg_overall, 
    ROUND(AVG(victories) OVER (PARTITION BY team), 2) AS avg_team, 
    RANK() OVER (ORDER BY victories DESC) AS rank_overall 
FROM drivers
WHERE victories IS NOT NULL 
ORDER BY rank_overall; 
'''
print("\nWindow function results (AVG, RANK):")
cur.execute(query_window_avg_rank)
for row in cur.fetchall(): print(row)

## Window Functions vs. `GROUP BY`

- A key difference: `GROUP BY` **reduces** the number of rows to one per group. Window functions **maintain** all original rows and add new columns based on the window calculation.

In [ ]:
# GROUP BY example (summarizes)
cur.execute("SELECT team, ROUND(AVG(victories), 2) FROM drivers WHERE victories IS NOT NULL GROUP BY team")
print("--- GROUP BY Output (Collapsed) ---")
for row in cur.fetchall(): print(row)

# Window Function example (adds detail to each row)
cur.execute('''
    SELECT driver_name, team, victories, 
           ROUND(AVG(victories) OVER (PARTITION BY team), 2) as avg_in_team
    FROM drivers 
    WHERE victories IS NOT NULL ORDER BY team, victories DESC
''')
print("\n--- Window Function Output (All Rows) ---")
for row in cur.fetchall(): print(row)

## Try it yourself! 🤓 {#sec:exercise02}

- Use a window function:
- Select `driver_name`, `nationality`, `victories`.
- Add `rank_nationality`: rank drivers by `victories` **within** each `nationality`. (**`PARTITION BY`** needed).
- Exclude `NULL` victories.
- Order by `nationality`, then `rank_nationality`.
- Use `cur.execute()` and fetching/looping.

In [ ]:
# Your SQL query here for Exercise 02
query_rank_nationality_exercise = '''
SELECT 
    driver_name, 
    nationality, 
    victories,
    RANK() OVER (PARTITION BY nationality ORDER BY victories DESC) AS rank_nationality
FROM drivers
WHERE victories IS NOT NULL 
ORDER BY nationality, rank_nationality; 
'''

# print("\n--- Drivers ranked by victories within nationality (Exercise) ---")
# cur.execute(query_rank_nationality_exercise)
# for row in cur.fetchall(): 
#     print(row)

# String Manipulation

## Basic String Functions

- SQLite provides standard functions for text processing.
- `LENGTH(str)`, `UPPER(str)`, `LOWER(str)`.
- `SUBSTR(str, start, length)` extracts a portion of the string (**start is 1-indexed**).

In [ ]:
query_str1 = '''
SELECT driver_name, 
    LENGTH(driver_name) AS len,
    UPPER(driver_name) AS upper,
    LOWER(driver_name) AS lower,
    SUBSTR(driver_name, 1, 4) AS first_four -- Get first 4 characters
FROM drivers LIMIT 4;
'''
print("\nString function examples (Part 1):")
cur.execute(query_str1)
for row in cur.fetchall(): print(row)

## More String Functions

- `TRIM(str)` removes leading/trailing whitespace.
- `REPLACE(str, find, replace)` substitutes text.
- `INSTR(str, find)` finds the starting position of `find` within `str` (**0 if not found**).
- The **`||`** operator concatenates (joins) strings.

In [ ]:
query_str2 = '''
SELECT driver_name, 
    REPLACE(driver_name, ' ', '_') AS replaced_space,
    INSTR(LOWER(driver_name), 'a') AS first_a_pos, 
    'Driver: ' || driver_name AS labelled_name -- Concatenation
FROM drivers LIMIT 4;
'''
print("\nString function examples (Part 2):")
cur.execute(query_str2)
for row in cur.fetchall(): print(row)

# Conditional Logic: CASE

## Basic `CASE` Statement

- The `CASE` statement is SQL's way of implementing **if-then-else logic** within a query.
- `CASE WHEN condition THEN result ELSE default_result END`

In [ ]:
query_case1 = '''
SELECT driver_name, victories,
    CASE 
        WHEN victories > 50 THEN 'Legend' 
        ELSE 'Great Driver (or N/A)'      
    END AS category 
FROM drivers;
'''
print("\nCASE statement example (Basic):")
cur.execute(query_case1)
for row in cur.fetchall(): print(row)

## `CASE` with Multiple Conditions

- You can chain multiple `WHEN` clauses. The **first condition that evaluates to true** determines the result for that row. The optional `ELSE` handles cases where no `WHEN` is true.

In [ ]:
query_case2 = '''
SELECT driver_name, victories,
    CASE 
        WHEN victories > 100 THEN 'All-Time Great'
        WHEN victories > 50 THEN 'Legend'
        WHEN victories >= 10 THEN 'Race Winner'
        WHEN victories > 0 THEN 'Podium Potential'
        ELSE 'Data Missing or Zero Wins' 
    END AS status
FROM drivers ORDER BY victories DESC NULLS LAST;
'''
print("\nCASE statement example (Multiple Conditions):")
cur.execute(query_case2)
for row in cur.fetchall(): print(row)

## `CASE` for Conditional NULL Handling

- `CASE` offers more nuanced control over handling `NULL`s compared to just `COALESCE`, allowing checks on **other columns**.

In [ ]:
query_case_null = '''
SELECT driver_name, 
    -- Fill nationality based on name if NULL
    CASE 
        WHEN nationality IS NULL AND driver_name = 'Fernando Alonso' THEN 'Spanish'
        WHEN nationality IS NULL THEN 'Unknown' 
        ELSE nationality
    END AS nationality_filled,
    -- Fill victories based on team if NULL
    CASE 
        WHEN victories IS NULL AND team = 'Aston Martin' THEN 32 -- Educated guess! 
        WHEN victories IS NULL THEN 0 
        ELSE victories
    END AS victories_filled
FROM drivers WHERE driver_name LIKE 'F%' OR driver_name LIKE 'L%'; -- Limit output
'''
print("\nCASE statement for conditional NULL handling:")
cur.execute(query_case_null)
for row in cur.fetchall(): print(row)

## Try it yourself! 🤓 {#sec:exercise03}

- Use `CASE` to create `driver_level`:
  - 'Beginner' if victories < 10 (and not NULL)
  - 'Intermediate' if 10 <= victories <= 50
  - 'Expert' if victories > 50
  - 'Unknown' if victories is NULL (**Handle NULL first!**)
- Select `driver_name`, `victories`, `driver_level`.
- Use `cur.execute()` and fetching/looping.

In [ ]:
# Your SQL query here for Exercise 03
query_driver_level_exercise = '''
SELECT 
    driver_name, 
    victories,
    CASE 
        WHEN victories IS NULL THEN 'Unknown' 
        WHEN victories > 50 THEN 'Expert'
        WHEN victories >= 10 THEN 'Intermediate' -- Corrected from 'BETWEEN 10 AND 50' to cover exactly 10 and 50
        WHEN victories < 10 THEN 'Beginner' -- This will catch >0 and <10
        ELSE 'Other' -- Should not be reached if NULL is handled first
    END AS driver_level
FROM drivers
ORDER BY victories DESC NULLS LAST; 
'''

# print("\n--- Driver classification using CASE (Exercise) ---")
# cur.execute(query_driver_level_exercise)
# for row in cur.fetchall(): 
#     print(row)

# Using SQL with `pandas` 🐼

## Reading Data with `pandas.read_sql`

- Reading data using loops and `fetchall()` works, but `pandas` makes it much easier for analysis.
- **`pandas.read_sql()`** executes a `SELECT` query and loads the results directly into a DataFrame.

In [ ]:
import pandas as pd # Import pandas

# Pass the SQL query and the connection object
df = pd.read_sql('SELECT * FROM drivers', conn) 
print("DataFrame loaded from SQL:")
display(df) 

- This combines execution and fetching into one step, returning a structured table.

## `read_sql` with Any `SELECT` Query

- You can use complex `SELECT` statements, including joins, filtering, ordering, etc., within `read_sql`.

In [ ]:
query_pd = """
SELECT driver_name, team, victories 
FROM drivers 
WHERE victories > 5 OR victories IS NULL
ORDER BY team
"""
df_filtered_ordered = pd.read_sql(query_pd, conn)
print("\nFiltered and ordered DataFrame from SQL:")
display(df_filtered_ordered)

- `pandas` handles retrieving all the data specified by your SQL query.

## Manipulating DataFrames from SQL

- Once loaded, the DataFrame behaves like any other `pandas` DataFrame. Apply all your data manipulation skills!

In [ ]:
# df was loaded previously
if 'df' in locals() and not df.empty:
    avg_vic_pd = df.groupby('team')['victories'].mean().dropna()
    print("\nAverage victories by team (Pandas):")
    print(avg_vic_pd)
else:
    print("DataFrame 'df' not available.")

In [ ]:
# Example using pandas .query() method for filtering
if 'df' in locals() and not df.empty:
    print("\nPandas query on DataFrame:")
    display(df.query('victories > 50 and nationality == "British"'))
else:
    print("DataFrame 'df' not available.")

## Writing DataFrames to SQL (`to_sql`)

- The reverse is also easy: `DataFrame.to_sql()` writes the DataFrame's contents into a database table.

In [ ]:
# Example: Create a DataFrame with British drivers
if 'df' in locals() and not df.empty:
    df_british = df[df['nationality'] == 'British'].copy()

    # Write to a new table named 'british_drivers'
    # index=False: [Important!] Prevents pandas index from becoming a DB column.
    df_british.to_sql('british_drivers', conn, if_exists='replace', index=False)
    print("\n'british_drivers' table created/replaced in DB.")

    # Verify by reading it back using pandas
    print("\nContents of 'british_drivers' table from DB:")
    display(pd.read_sql('SELECT * FROM british_drivers', conn))
else:
    print("DataFrame 'df' not available to create 'british_drivers'.")

## Try it yourself! 🤓 {#sec:exercise04}

- 1. Create `employees` table (`employee_id` INTEGER PRIMARY KEY AUTOINCREMENT, `employee_name` TEXT, `department` TEXT, `salary` INTEGER) using `cursor.execute()`.
- 2. Insert data for Alice (HR, 50k), Bob (IT, 60k), Charlie (HR, 70k), David (IT, 80k) using `cursor.executemany()` or multiple `cursor.execute()` calls. Commit the changes.
- 3. Read the full `employees` table into a pandas DataFrame called `df_employees` using **`pd.read_sql()`**. Print `df_employees`.
- 4. Using **`df_employees`**, calculate and print the average salary per department.

In [ ]:
# Your code here for Exercise 04

# import pandas as pd
# import sqlite3

# db_file_ex4 = 'exercise04.db' # Use a new db file or the existing lecture12.db
# conn_ex4 = sqlite3.connect(db_file_ex4)
# cur_ex4 = conn_ex4.cursor()

# 1. Create employees table
# cur_ex4.execute('DROP TABLE IF EXISTS employees;') 
# cur_ex4.execute('''
# CREATE TABLE employees (
#     employee_id INTEGER PRIMARY KEY AUTOINCREMENT,
#     employee_name TEXT, 
#     department TEXT, 
#     salary INTEGER
# );''')

# 2. Insert data
# employee_data = [
#     ('Alice', 'HR', 50000), ('Bob', 'IT', 60000),
#     ('Charlie', 'HR', 70000), ('David', 'IT', 80000)
# ]
# cur_ex4.executemany('INSERT INTO employees (employee_name, department, salary) VALUES (?, ?, ?)', employee_data)
# conn_ex4.commit()
# print("Employees table created and populated.")

# 3. Read data with pandas
# df_employees = pd.read_sql('SELECT * FROM employees', conn_ex4)
# print("\n--- Employees DataFrame ---")
# display(df_employees)

# 4. Compute average salary by department using pandas
# if not df_employees.empty:
#     avg_salary_dept = df_employees.groupby('department')['salary'].mean()
#     print("\n--- Average salary by department (pandas) ---")
#     print(avg_salary_dept)

# conn_ex4.close() # Close the connection for this exercise

# Pivot tables in SQL 📄

## Pivot Setup: Student Scores

- Pivoting reshapes data from "long" to "wide" format. SQLite lacks a dedicated `PIVOT` command, so we use conditional aggregation.
- Let's set up the `student_scores` data first.

In [ ]:
# Create and populate student_scores table
cur.execute('DROP TABLE IF EXISTS student_scores;')
cur.execute('CREATE TABLE student_scores (student_name TEXT, subject TEXT, score INTEGER, term TEXT);')
scores_data = [
    ('Alice', 'Maths', 90, 'Q1'), ('Alice', 'Maths', 80, 'Q2'), ('Alice', 'Science', 80, 'Q1'), ('Alice', 'Science', 75, 'Q2'),
    ('Bob', 'Maths', 80, 'Q1'), ('Bob', 'Maths', 100, 'Q2'), ('Bob', 'Science', 80, 'Q1'), ('Bob', 'Science', 70, 'Q2'),
    ('Charles', 'Maths', 70, 'Q1'), ('Charles', 'Maths', 75, 'Q2'), ('Charles', 'Science', 90, 'Q1'), ('Charles', 'Science', 85, 'Q2')
]
cur.executemany('INSERT INTO student_scores VALUES (?, ?, ?, ?)', scores_data)
conn.commit()
print("Created and populated student_scores.")

# Display raw data using pandas
print("\nRaw student_scores data:")
display(pd.read_sql('SELECT * FROM student_scores ORDER BY student_name, term, subject', conn))

## Pivoting with `CASE` and `GROUP BY`

- The technique involves using an aggregate function (`AVG`, `SUM`, etc.) around a `CASE` statement for each desired new column. **`GROUP BY`** ensures one row per student.

In [ ]:
pivot_query = '''
SELECT 
    student_name,
    -- Avg Maths: Only average score WHEN subject is Maths
    AVG(CASE WHEN subject = 'Maths' THEN score END) as Avg_Maths, 
    -- Avg Science: Only average score WHEN subject is Science
    AVG(CASE WHEN subject = 'Science' THEN score END) as Avg_Science
FROM student_scores
GROUP BY student_name 
ORDER BY student_name;
'''
# Display the pivoted result using pandas
df_pivot = pd.read_sql(pivot_query, conn)
print("\nPivoted student scores (Average by subject):")
display(df_pivot)

- **`AVG` ignores NULLs**, which are produced by the `CASE` statement when the subject doesn't match.

# Conclusion 📚

## Conclusion

- `sqlite3` provides the basic tools to interact with SQLite from Python.
- Key steps: `connect()`, `cursor()`, `execute()`, **`commit()`**.
- Fetch results using `fetchone()`, `fetchall()`, or by iterating the cursor.
- Common SQL commands work as expected.
- Window functions require **modern SQLite versions**.
- Use `CASE` + `GROUP BY` for pivoting.
- **`pandas.read_sql()`** greatly simplifies loading data for analysis.
- `df.to_sql()` saves DataFrames back to the database.

![](figures/pandasql5.jpg)
Source: [Susan Ibach (2020)](https://susanibach.wordpress.com/2020/01/07/pandas-for-sql-lovers-select-from-table/)

# And that's all for today! 🚀

# Have a great day! 😊

In [ ]:
# Clean up: Close the connection
if 'conn' in locals() and conn:
    conn.close()
    print("\nSQLite connection closed.")